In [126]:
import numpy as np
import pandas as pd
import regex as re
import os
import bs4
from bs4 import BeautifulSoup

In [127]:
directory = "downloaded"
i = 0
for filename in os.listdir(directory):
    filename2 = filename.replace(".html","")
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        with open(f, "r") as read_file:
            content = read_file.read()

        if not content.__contains__("Learning Objectives"):
            print(filename + " has nothing")
            #move it to the "no objectives" folder
            os.rename(f, f"downloaded/no objectives/{filename}")
            continue

        # if filename2 != "writingforsuccess":
        #     continue

        print("\n\n\n\n\n\n\n\n\n" + filename + " contains Learning Objectives")

        soup = BeautifulSoup(content, 'html.parser')

        # Remove all exercises            
        for el in soup.find_all("div", {"class": "textbox--exercises"}):
            el.decompose()
        for el in soup.find_all("div", {"class": "review-questions"}):
            el.decompose()
        for el in soup.find_all("div", {"class": "critical-thinking"}):
            el.decompose()
        for el in soup.find_all("div", {"class": "exercise"}):
            el.decompose()
        for el in soup.find_all("div", {"title": "Mid-Chapter Questions"}):
            el.decompose()
        for el in soup.find_all("div", {"title": "End of Chapter Questions"}):
            el.decompose()

        # If an element contains "attributions", remove it and the next element
        for el in soup.find_all("h2", text=re.compile("Attributions")):
            el.decompose()
            if el.next_sibling:
                el.next_sibling.decompose()
        for el in soup.find_all("h3", text=re.compile("Attributions")):
            el.decompose()
            if el.next_sibling:
                el.next_sibling.decompose()
        for el in soup.find_all("h3", text=re.compile("Exercises and Critical Thinking")):
            el.decompose()
            if el.next_sibling:
                el.next_sibling.decompose()
        
        for el in soup.find_all("h3", text=re.compile("Supplemental Exercises")):
            el.parent.decompose()

        # If an h3 contains "Section Quiz", remove it and its parent
        for el in soup.find_all("h3", text=re.compile("Section Quiz")):
            el.parent.decompose()

        for el in soup.find_all("h3", text=re.compile("Self-practice")):
            el.parent.decompose()
        for el in soup.find_all("h3", text=re.compile("Self-Practice")):
            el.parent.decompose()


        for el in soup.find_all("strong", text=re.compile("H5P")):
            # print("el.name: " + el.name + " el.parent.name: " + el.parent.name + " el.parent.parent.name: " + el.parent.parent.name)
            if el.parent is not None:
                if el.parent.name != "div":
                    el.parent.parent.decompose()
                else:
                    el.parent.decompose()
            else:
                el.decompose()
            
        for el in soup.find_all("strong", text=re.compile("Practice Exercise")):
            # print("el.name: " + el.name + " el.parent.name: " + el.parent.name + " el.parent.parent.name: " + el.parent.parent.name)
            if el.parent is not None:
                if el.parent.name != "div":
                    el.parent.parent.decompose()
                else:
                    el.parent.decompose()
            else:
                el.decompose()  

        for el in soup.find_all("strong", text=re.compile("Practice EXERCISE")):
            # print("el.name: " + el.name + " el.parent.name: " + el.parent.name + " el.parent.parent.name: " + el.parent.parent.name)
            if el.parent is not None:
                if el.parent.name != "div":
                    el.parent.parent.decompose()
                else:
                    el.parent.decompose()
            else:
                el.decompose()  
                 



        if filename2 == "biology":
            for el in soup.find_all("ol"):
                el.decompose()




        content = soup.prettify()
        
        #remove all \n
        content = re.sub('\n', '', str(content))

        #remove all &nbsp;
        content = re.sub('&nbsp;', '', str(content))

        #remove all the special characters like &#8220;
        content = re.sub(r'&#\d+;', '', str(content))

        #add space after list elements
        content = re.sub('</li>', '</li>\n', str(content))
        
        #remove all the html tags
        content = re.sub('<[^<]+?>', '', str(content))

        #trim each line
        content = re.sub(r'^\s+', '', str(content), flags=re.MULTILINE)            

        #remove all the duplicate spaces
        content = re.sub(r'(\s+)', lambda x:x.group(0), content, flags=re.DOTALL)

        #remove all the broken content
        content = re.sub(r'An interactive H5P element has been excluded from this version of the text. You can view it online here:', '', str(content))

        #remove all the links
        content = re.sub(r'https?://\S+', '', str(content))


        sections = content.split("Learning Objectives")
        lectures=[]
        questions=[]
        for (sectionidx,section) in enumerate(sections[1:]):
            #the first few lines are the learning objectives. as soon as a line has one of the following, it is part of the lecture:
            # more than 200 characters
            # a © symbol 
            # it contains the text "Media Attributions"
            # it contains "Figure \d"
            temp = section.split("\n")
            question = ""
            lecture = ""
            processing_lec = False
            #in the first line, remove everything that comes before the first colon
            temp[0] = re.sub(r'^.*?:', '', temp[0])
            for t in temp:
                # print("processing: " + t[:200])
                if len(t) > 200 or t.__contains__("©") or t.__contains__("Media Attributions") or re.search(r'Figure \d', t):
                    processing_lec = True
                if processing_lec:
                    lecture += t
                else:
                    question += t+"\n"
            
            #trim the question
            question = re.sub(r'^\s+', '', question, flags=re.MULTILINE)
            question = re.sub(r'\s+$', '', question, flags=re.MULTILINE)
            #remove duplicate spaces from the lecture
            lecture = re.sub(r'([ ]+)', ' ', lecture, flags=re.DOTALL)
            #remove duplicate spaces from the question
            question = re.sub(r'([ ]+)', ' ', question, flags=re.DOTALL)
            if len(question) > 0:
                lectures.append(lecture)
                questions.append(question)

            # print("question: " + question)
            # print("lecture: " + lecture[:500])
            # break



        print("extracted " + str(len(lectures)) + " lectures and " + str(len(questions)) + " questions")
        for i in range(min(3, len(lectures))):
            print("QUESTIONS " + str(i))
            print(re.sub(r'\n', ' ### ', questions[i][:1000], flags=re.MULTILINE))
            print("LECTURE " + str(i))
            print(re.sub(r'\n', '', lectures[i][:1000], flags=re.MULTILINE))
            print(re.sub(r'\n', '', lectures[i][-1000:], flags=re.MULTILINE))
            print("")



        if filename2 != "biology":
            blacklist = ["____", "Exercises","Questions","Guided Practice", "Explore More", "Review Questions", "Which of the following", "Test Yourself", "Self-practice"]
            strings = []
            for lecture in lectures:
                if any(lecture.__contains__(word) for word in blacklist):
                    
                    occ = [lecture.find(end) for end in blacklist]
                    idx = np.argmin([999999999 if o == -1 else o for o in occ])
                    loc = occ[idx]
                    strings.append(lecture[loc-50:loc+250])

            if len(strings) > 4:
                print("found too many blacklisted strings")
                for s in strings:
                    print(s)



        if len(lectures) == 1:
            print("only one set of objectives")
            os.rename(f, f"downloaded/no objectives/{filename}")
            continue
        else:
            filename2 = filename.replace(".html","")
            location = f"processed_learning_objectives/{filename2}.pq"
            # create a dataframe that contains all the questions and lectures separately
            df = pd.DataFrame(columns=["Chapter", "Text", "is_question", "is_learning_objective"])
            for i in range(len(lectures)):
                df = pd.concat([df, pd.DataFrame([[i, lectures[i], 0, 0]], columns=["Chapter", "Text", "is_question", "is_learning_objective"])])
                for q in questions[i].split("\n"):
                    if len(q) > 0:
                        df = pd.concat([df, pd.DataFrame([[i, q, 0, 1]], columns=["Chapter", "Text", "is_question", "is_learning_objective"])])
                

            # print(df)
            df.to_parquet(location, index=False)
            # break
            continue















biology.html contains Learning Objectives
extracted 100 lectures and 100 questions
QUESTIONS 0
Identify and describe the properties of life ### Describe the levels of organization among living things ### List examples of different sub disciplines in biology
LECTURE 0
Watch a video about Evolution by Natural Selection. Biology is the science that studies life. What exactly is life? This may sound like a silly question with an obvious answer, but it is not easy to define life. For example, a branch of biology called virology studies viruses, which exhibit some of the characteristics of living entities but lack others. It turns out that although viruses can attack living organisms, cause diseases, and even reproduce, they do not meet the criteria that biologists use to define life. From its earliest beginnings, biology has wrestled with four questions: What are the shared properties that make something “alive”? How do those various living things function? When faced with the rema

In [128]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(pd.read_parquet("./processed_learning_objectives/biology.pq"))

     Chapter                                               Text  is_question  \
0          0  Watch a video about Evolution by Natural Selec...            0   
1          0       Identify and describe the properties of life            0   
2          0  Describe the levels of organization among livi...            0   
3          0  List examples of different sub disciplines in ...            0   
4          1  Watch a video about the Scientific Method. Fig...            0   
5          1  Identify the shared characteristics of the nat...            0   
6          1       Understand the process of scientific inquiry            0   
7          1  Compare inductive reasoning with deductive rea...            0   
8          1  Describe the goals of basic science and applie...            0   
9          2  Watch a video about electrons and how the elec...            0   
10         2                       Describe matter and elements            0   
11         2  Describe the interrelation